In [89]:
import json
import pandas as pd

# json 파일 읽어 가져오기
with open('/Users/jaynam/workspace/airflow/data/subway_data_20221225.json', 'r') as sw_json:
    sw_json_data = json.load(sw_json)


sub_list = ['연번', '역번호', '역명', '구분', '호선', '조사일자']

time_list = ['10시00분', '10시30분', 
             '11시00분', '11시30분',
             '12시00분', '12시30분',
             '13시00분', '13시30분',
             '14시00분', '14시30분',
             '15시00분', '15시30분',
             '16시00분', '16시30분',
             '17시00분', '17시30분',
             '18시00분', '18시30분',
             '19시00분', '19시30분',
             '20시00분', '20시30분',
             '21시00분', '21시30분',
             '22시00분', '22시30분',
             '23시00분', '23시30분',          
             '5시30분', 
             '6시00분', '6시30분', 
             '7시00분', '7시30분', 
             '8시00분', '8시30분', 
             '9시00분', '9시30분' ]

def get_sw_ref(sw_df):
    info_df = sw_df[sub_list]
    time_df = sw_df[time_list]
    
    # 행열 바꾸기
    time_rev = time_df.transpose()

    # 컬럼 추가
    time_rev.rename(columns = {0 : '혼잡도'}, inplace = True)
    time_rev['시간'] = time_df.columns.values
    time_rev['연번'] = info_df['연번'][0]

    # 컬럼 순서 변경
    time_rfm = time_rev[['연번', '시간', '혼잡도']]

    # 인덱스 설정
    time_res = time_rfm.set_index('연번')
    
    # 연번을 기준으로 merge
    merge_df = pd.merge(left = info_df , right = time_res, how = "outer", on = "연번")

    return merge_df
    

sw_result = pd.DataFrame()

for i in range(0, len(sw_json_data['data'])):
    sw_df = pd.json_normalize(sw_json_data['data'][i])
    tmp_df = get_sw_ref(sw_df)
    
    sw_result = pd.concat([sw_result, tmp_df])


sw_result

,연번,역번호,역명,구분,호선,조사일자,시간,혼잡도
0,1,150,서울역,상선,1,평일,10시00분,26.6
1,1,150,서울역,상선,1,평일,10시30분,23.4
2,1,150,서울역,상선,1,평일,11시00분,22.4
3,1,150,서울역,상선,1,평일,11시30분,22.1
4,1,150,서울역,상선,1,평일,12시00분,21.5
...,...,...,...,...,...,...,...,...
32,1704,2827,모란,하선,8,일요일,7시30분,0.0
33,1704,2827,모란,하선,8,일요일,8시00분,0.0
34,1704,2827,모란,하선,8,일요일,8시30분,0.0
35,1704,2827,모란,하선,8,일요일,9시00분,0.0


In [82]:
sw_result[(sw_result['시간']=='19시00분')&(sw_result['구분']=='상선')]

,연번,역번호,역명,구분,호선,조사일자,시간,혼잡도
18,1,150,서울역,상선,1,평일,19시00분,20.0
18,3,151,시청,상선,1,평일,19시00분,24.1
18,5,152,종각,상선,1,평일,19시00분,28.8
18,7,153,종로3가,상선,1,평일,19시00분,29.6
18,9,154,종로5가,상선,1,평일,19시00분,34.3
...,...,...,...,...,...,...,...,...
18,1695,2823,남한산성입구,상선,8,일요일,19시00분,6.1
18,1697,2824,단대오거리,상선,8,일요일,19시00분,5.8
18,1699,2825,신흥,상선,8,일요일,19시00분,4.9
18,1701,2826,수진,상선,8,일요일,19시00분,4.6


In [94]:
out_df = sw_result[(sw_result['시간']=='19시00분')&(sw_result['구분']=='상선')]

out2 = out_df[['연번', '역명', '혼잡도']]

out2['혼잡도'].plot()

TypeError: no numeric data to plot